# Neurosciences Papers 2022 - 2023

In [12]:
# Import Libraries

import sys
import requests
from tqdm.notebook import tqdm
from pymongo import MongoClient

sys.path.append('../py')
from src import *

## DB creation with Mongo

In [17]:
str_conn = 'mongodb://localhost:27017/'
cursor = MongoClient(str_conn)
cursor

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True)

In [23]:
db = cursor.publications

In [24]:
colec = db.biorxiv
colec

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'publications'), 'biorxiv')

## DB feeding with API

![biorxiv.png](attachment:biorxiv.png)

To select just the neuroscience papers I will follow the next #:

https://api.biorxiv.org/details/[server]/[interval]/[cursor]/[format]

    server = biorxiv
    interval = ['2022-01-01/2022-12-31', '2023-01-01/2023-12-31']
    cursor = [0, len(pages)]
    format = JSON

In [25]:
interval = ['2022-01-01/2022-12-31', 
            '2023-01-01/2023-12-31']

In [32]:
list_urls = [f'https://api.biorxiv.org/details/biorxiv/{i}/{j}/json' 
             for j in range(10000) for i in interval]

* The first loop will iterate the intervals for the url, that are 2022 and 2023.
* The second loop will iterate the number of pages.
* The third loop will insert each article in the database.

The API present sometimes the same article multiple times,
to avoid duplicates, I will use the DOI as id_.

In [33]:
%%time

for url in tqdm(list_urls[13720:]):
    
    res = requests.get(url)
    res_api = res.json()   

    for k in range(100):
        if res_api['collection'][k]['category'] == 'neuroscience':
            articulo = res_api['collection'][k]
            doi = articulo.pop("doi")
            articulo["_id"] = doi
            try:
                colec.insert_one(articulo)
            except:
                continue
        else:
            continue  

  0%|          | 0/10000 [00:00<?, ?it/s]

ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))

    2022 total # of articles: 35750
    2023 total # of articles: 48588